In [1]:
import numpy as np
import os
import sys

from keras.callbacks import (TensorBoard,
                             EarlyStopping,
                             ReduceLROnPlateau,
                             CSVLogger)
from keras.layers import Conv3D, Dense, Reshape, Add, Input
from keras.models import Sequential, Model
from keras.optimizers import SGD, Adam
from keras import backend as K
from keras.utils import to_categorical, multi_gpu_model
from keras.utils.vis_utils import plot_model
import matplotlib.pyplot as plt
import tensorflow as tf

from capsulenet import margin_loss
from capsulelayers import CapsuleLayer, PrimaryCap, Length, Mask, Conv3DCap

from data import load_data
from utils import upsample_classes, stratified_shuffle
from results import process_results


NAME= 'ModelNet40'
NUM_EPOCHS = 25

# Load the data
(x_train, y_train), (x_test, y_test), target_names = load_data(NAME)
x_train, y_train, x_val, y_val = stratified_shuffle(x_train, y_train, test_size=.1)
x_train, y_train = upsample_classes(x_train, y_train)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_val = to_categorical(y_val)
x_train, y_train, x_val, y_val, x_test, y_test = \
    x_train[:256], y_train[:256], x_val[:100], y_val[:100], x_test[:100], y_test[:100]
NUM_EPOCHS = 1

n_class = y_test.shape[1]
input_shape = (30, 30, 30, 1)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
dim_sub_capsule=16
dim_primary_capsule=5
n_channels=8
primary_cap_kernel_size=9
first_layer_kernel_size=9
conv_layer_filters=48

In [5]:
x = Input(shape=input_shape)
conv1 = Conv3D(filters=conv_layer_filters, kernel_size=3, strides=1,
                              padding='valid', activation='relu', name='conv1')(x)
primarycaps = Conv3DCap(conv1, dim_capsule=4, n_channels=4,
                        kernel_size=3, strides=2, padding='valid',
                        name='primarycap_conv3d')
secondary_caps = PrimaryCap(primarycaps, dim_capsule=4, n_channels=4,
                            kernel_size=2, strides=1, padding='valid',
                            name='secondarycap_conv3d')
sub_caps = CapsuleLayer(num_capsule=n_class, dim_capsule=dim_sub_capsule,
                                                 routings=3, name='sub_caps')(secondary_caps)

out_caps = Length(name='capsnet')(sub_caps)

# Decoder network
y = Input(shape=(n_class,))
masked_by_y = Mask()([sub_caps, y])
masked = Mask()(sub_caps)

# shared decoder model in training and prediction
decoder = Sequential(name='decoder')
decoder.add(Dense(512, activation='relu',
                  input_dim=dim_sub_capsule*n_class))
decoder.add(Dense(1024, activation='relu'))
decoder.add(Dense(np.prod(input_shape), activation='sigmoid'))
decoder.add(Reshape(target_shape=input_shape, name='out_recon'))


### Models for training and evaluation (prediction and actually using)
train_model = Model([x, y], [out_caps, decoder(masked_by_y)])
eval_model = Model(x, [out_caps, decoder(masked)])

### manipulate model can be used to visualize activation maps for specific classes
noise = Input(shape=(n_class, dim_sub_capsule))
noised_sub_caps = Add()([sub_caps, noise])
masked_noised_y = Mask()([noised_sub_caps, y])
manipulate_model = Model([x, y, noise], decoder(masked_noised_y))

In [6]:
INIT_LR = 0.008
lam_recon = .04
optimizer = Adam(lr=INIT_LR)
train_model.compile(optimizer,
                    loss=[margin_loss, 'mse'],
                    loss_weights=[1., lam_recon],
                    metrics={'capsnet': 'accuracy'})

In [7]:
train_model.fit([x_train, y_train], [y_train, x_train],
                                batch_size=32, epochs=NUM_EPOCHS,
                                validation_data=[[x_val, y_val], [y_val, x_val]])

Train on 256 samples, validate on 100 samples
Epoch 1/1
256/256 [==============================] - 29s 113ms/step - loss: 0.8200 - capsnet_loss: 0.8100 - decoder_loss: 0.2492 - capsnet_acc: 0.2773 - val_loss: 0.8199 - val_capsnet_loss: 0.8100 - val_decoder_loss: 0.2477 - val_capsnet_acc: 0.0800


In [7]:
import numpy as np
import os
import sys

from keras.callbacks import (TensorBoard,
                             EarlyStopping,
                             ReduceLROnPlateau,
                             CSVLogger)
from keras.layers import Conv3D, Dense, Reshape, Add, Input
from keras.models import Sequential, Model
from keras.optimizers import SGD, Adam
from keras import backend as K
from keras.utils import to_categorical, multi_gpu_model
from keras.utils.vis_utils import plot_model
import matplotlib.pyplot as plt
import tensorflow as tf

from capsulenet import margin_loss
from capsulelayers import CapsuleLayer, PrimaryCap, Length, Mask

from data import load_data
from utils import upsample_classes, stratified_shuffle
from results import process_results


NAME= 'ModelNet40'
NUM_EPOCHS = 25

# Load the data
(x_train, y_train), (x_test, y_test), target_names = load_data(NAME)
x_train, y_train, x_val, y_val = stratified_shuffle(x_train, y_train, test_size=.1)
x_train, y_train = upsample_classes(x_train, y_train)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_val = to_categorical(y_val)
x_train, y_train, x_val, y_val, x_test, y_test = \
    x_train[:256], y_train[:256], x_val[:100], y_val[:100], x_test[:100], y_test[:100]
NUM_EPOCHS = 1

n_class = y_test.shape[1]
input_shape = (30, 30, 30, 1)

dim_sub_capsule=16
dim_primary_capsule=5
n_channels=8
primary_cap_kernel_size=9
first_layer_kernel_size=9
conv_layer_filters=48

x = Input(shape=input_shape)
conv1 = Conv3D(filters=conv_layer_filters, kernel_size=first_layer_kernel_size, strides=1,
                              padding='valid', activation='relu', name='conv1')(x)
primarycaps = PrimaryCap(conv1, dim_capsule=dim_primary_capsule, n_channels=n_channels,
                                                  kernel_size=primary_cap_kernel_size, strides=2, padding='valid',
                                                  name='primarycap_conv3d')
secondary_caps = PrimaryCap(primarycaps, dim_capsule=8, n_channels=4,
                            kernel_size=2, strides=1, padding='valid',
                            name='secondarycap_conv3d')
sub_caps = CapsuleLayer(num_capsule=n_class, dim_capsule=dim_sub_capsule,
                                                 routings=3, name='sub_caps')(secondary_caps)
# sub_caps = CapsuleLayer(num_capsule=n_class, dim_capsule=dim_sub_capsule,
#                                                  routings=3, name='sub_caps')(primarycaps)
out_caps = Length(name='capsnet')(sub_caps)

# Decoder network
y = Input(shape=(n_class,))
masked_by_y = Mask()([sub_caps, y])
masked = Mask()(sub_caps)

# shared decoder model in training and prediction
decoder = Sequential(name='decoder')
decoder.add(Dense(512, activation='relu',
                  input_dim=dim_sub_capsule*n_class))
decoder.add(Dense(1024, activation='relu'))
decoder.add(Dense(np.prod(input_shape), activation='sigmoid'))
decoder.add(Reshape(target_shape=input_shape, name='out_recon'))


### Models for training and evaluation (prediction and actually using)
train_model = Model([x, y], [out_caps, decoder(masked_by_y)])
eval_model = Model(x, [out_caps, decoder(masked)])

### manipulate model can be used to visualize activation maps for specific classes
noise = Input(shape=(n_class, dim_sub_capsule))
noised_sub_caps = Add()([sub_caps, noise])
masked_noised_y = Mask()([noised_sub_caps, y])
manipulate_model = Model([x, y, noise], decoder(masked_noised_y))